**Install all essential libraries**
* tensorflow is our general purpose ml library with all the utlities we need
* numpy has all our various data types like arrays, matrices etc.
* matplotlib for plotting and visualising the numpy array of values
* math is just for math functions and mathematical constants

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math

**Library diagnostic information**

In [ ]:
print('Numpy ' + np.__version__)
print('Tensorflow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)
print('Mathplotlib ' + matplotlib.__version__)

**Model Traning Parameters**
* training - to train the model.
* validation - to verify and tune the model.
* test - unseen data to see the performance of the model.

In [ ]:
nsamples = 1000
val_ratio = 0.2     # 20% of data is for validation
test_ratio = 0.2    # 20% of data is for testing, overall 600 samples for training
tflite_model_name = 'sine_model'
c_model_name = 'sine_model'

**generate some sample data**

In [ ]:
np.random.seed(123)
x_values = np.random.uniform(low=0, high=(2 * math.pi), size=nsamples)      # this will generate uniform distribution of random ness
plt.plot(x_values)

**Obtain output samples with noise** (for estimation)

In [ ]:
y_values = np.sin(x_values) + (0.05 * np.random.randn(x_values.shape[0]))    # add some noise just to make the model guess a little
plt.plot(x_values, y_values, '.')

**split the data across training, validation and test sets**

In [ ]:
val_split = int(val_ratio * nsamples)
test_split = int(val_split + (test_ratio * nsamples))
x_val, x_test, x_train = np.split(x_values, [val_split, test_split])
y_val, y_test, y_train = np.split(y_values, [val_split, test_split])

# check samples intact
assert(x_train.size + x_val.size + x_test.size == nsamples)

**show all the different data samples in the distribution**

In [ ]:
plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.plot(x_val, y_val, 'y.', label="Validate")

**Create the skeleton for the neural network model**

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))

In [ ]:
model.summary()

**Optimizer and Loss function and metrics**

In [ ]:
model.compile(optimizer='rmsprop', loss='mae', metrics=['mae'])

**Train the model**

In [ ]:
with tf.device('/CPU:0'):
  history = model.fit(x_train,
                    y_train,
                    epochs=500,
                    batch_size=100,
                    validation_data=(x_val, y_val),
                  )

**Observe training history**

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

**plot the predictions against actual values**

In [ ]:
predictions = model.predict(x_test)

plt.clf()
plt.title('Comparison of predictions to actual values')
plt.plot(x_test, y_test, 'g.', label='Actual')
plt.plot(x_test, predictions, 'b.', label='Prediction')
plt.legend()
plt.show()

**Convert and optimize it as a tflite model**

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)